In [ ]:
%pip install dwave-system==1.18.0
%pip install dwave-system[drivers] --extra-index-url https://pypi.dwavesys.com/simple
%pip install dimod==0.11.0
%pip install dwave_qbsolv

%pip install dwave-ocean-sdk>=6.3.0
%pip install mip==1.13.0
%pip install plotly==5.6.0
%pip install streamlit==1.9.2
%pip install tabulate==0.8.9

In [ ]:
import os
import time
import random


import networkx as nx
import matplotlib.pyplot as plt

from dwave_qbsolv import QBSolv
from dwave.system import DWaveSampler
import dimod
import hybrid

import MapGraph


In [ ]:
# Get D-Wave token
TOKEN = os.environ['DWAVE_APITOKEN']

# Problem spec
N_CARS = 5
N_ROUTES = 3
ROUTES = []  
STREETS = None

In [ ]:
# Create graph | street 
G_ROW = 4
G_COLUMN = 5 

G = MapGraph.make_graph(G_ROW, G_COLUMN)
MapGraph.add_traffic_weight(G)
MapGraph.add_time_travel(G)

pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, with_labels=True)
plt.show()

In [ ]:
# Assign random origin and destination to each car
origin_cars = [random.randint(0, G_COLUMN*G_ROW-1) for x in range(N_CARS)]
dest_cars = [random.randint(0, G_COLUMN*G_ROW-1) for x in range(N_CARS)]


# Three (diverse?) routes for each car
routes = []

for n_cars in range(N_CARS):
    paths = MapGraph.find_tre_shortest_paths(G, 
                                             origin_cars[n_cars], 
                                             dest_cars[n_cars], 
                                             overlap=5, 
                                             weight="time travel")
    for j in range(N_ROUTES):
        routes.append(paths[j])

In [ ]:

# MapGraph.print_map_graph_details(G)

origin_node = 3
destination_node = 13

# Three shortes paths from origin to destination
paths = MapGraph.find_tre_shortest_paths(G, origin_node, destination_node, overlap=5, weight="time travel")
path_edges = MapGraph.paths_edges(paths)
path_edges

In [ ]:
streets = G.edges()
routs_A_B = path_edges

In [ ]:
# classmethod BinaryQuadraticModel.from_qubo(Q: Mapping, offset: float = 0)
#     Q – Coefficients of a quadratic unconstrained binary optimization (QUBO) problem 
#         as a dict of form {(u, v): bias, ...}, where u, v, are binary-valued variables 
#         and bias is their associated coefficient.
#
#     offset (optional, default=0.0) – Constant offset applied to the model.

# QUBO
#   q_ij: car i takes route (= a list of segments) j

def create_cars_variables(n_cars: int, routes: list):
        cars_variables = []

        for i in range(n_cars):
                for j in range(len(routes)):
                        cars_variables.append(f'q{i}{j}')
        return cars_variables

def create_routes_variables(cars_variables: list[str], routes: list, n_routes: int = 3):
        cars_routes = {}

        # Fill routes for car in a cyclic fashion (modulo N_ROUTES = 3)
        for i in range(len(cars_variables)):
                cars_routes[cars_variables[i]] = routes[route_index]


cars_vars = create_cars_variables(N_CARS, ROUTES)

cars_routes = create_routes_variables(cars_vars, routes, N_ROUTES)


